In [2]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.embeddings import HuggingFaceEmbeddings 
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader # For loading documents
from langchain.text_splitter import RecursiveCharacterTextSplitter # For splitting text (corpus) into smaller chunks
from langchain_core.prompts import ChatPromptTemplate
from langchain.llms import CTransformers # Using ctransfomers since we are using a quantized model
import os

In [3]:
# Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob = "*.pdf",
                    loader_cls = PyPDFLoader)
    
    documents = loader.load()
    return documents

In [ ]:
extract_data = load_pdf("data/")

In [ ]:
extract_data

In [ ]:
# Split the text into smaller chunks

def text_splitter(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    split_data = splitter.split_documents(data)
    return split_data

In [ ]:
split_data = text_splitter(extract_data)
print("The length of the chunks is:", len(split_data))

In [ ]:
# Donwload embedding model
def download_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_embedding()

In [ ]:
embeddings

In [ ]:
# Sample conversion of text to embeddings

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
query_result

In [ ]:
os.environ['PINECONE_API_KEY'] = "d0ccc28f-3d45-4cff-989a-ff83b043add8"

index_name = "medical-chatbot"

vectorstore = PineconeVectorStore(embedding = embeddings, index_name = index_name)

# For creating a sementic index if it does not exist.
# vectorstore = PineconeVectorStore.from_documents(split_data, embeddings, index_name = index_name)

In [ ]:
query = "What is the treatment for diabetes?"

similar_docs = vectorstore.similarity_search(query)
print(similar_docs)

In [ ]:
retriever = vectorstore.as_retriever()
print(type(retriever))


In [ ]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type = "llama",
                    config = {'max_new_tokens': 512,
                              'temperature': 0.8})

In [ ]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_template), 
        ("human", "{input}")
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, PROMPT)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
result = chain.invoke({"input": "What is acne?"})
print("Response:", result["answer"])